In [11]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
from sklearn.metrics import ndcg_score
import pandas as pd

In [41]:
!pip install -U catboost

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: catboost
    Found existing installation: catboost 1.0.4
    Uninstalling catboost-1.0.4:
      Successfully uninstalled catboost-1.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mljar-supervised 0.11.1 requires category-encoders>=2.2.2, but you have category-encoders 1.3.0 which is incompatible.
mljar-supervised 0.11.1 requires lightgbm>=3.0.0, but you have lightgbm 2.3.1 which is incompatible.
mljar-supervised 0.11

In [2]:
from catboost.datasets import msrank_10k
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
queries_train = train_df[1].values

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
queries_test = test_df[1].values

In [44]:
import json
with open('../dataset/ranking_train.jsonl') as f:
    train_data = [json.loads(line) for line in f]

In [45]:
post_texts = []
comments_texts = []
total_texts = []
scores = []
groups = []
for i,example in enumerate(train_data):
    for comment in example['comments']:
        post_texts.append(example['text'])
        comments_texts.append(comment['text'])
        total_texts.append(example['text'] + ' COMMENT: ' + comment['text'])
        scores.append(comment['score'])
        groups.append(i)
scores = 1.0-np.array(scores)/4
train_dataframe = pd.DataFrame([post_texts,comments_texts]).T

In [5]:
train = Pool(
    data=train_dataframe[:300000],
    label=scores[:300000],
    group_id=groups[:300000],
    text_features = [0,1]
)

test = Pool(
    data=train_dataframe[300000:],
    label=scores[300000:],
    group_id=groups[300000:],
    text_features = [0,1]
)

In [6]:
default_parameters = {
    'iterations': 1000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': True,
    'random_seed': 0,
}

parameters = {}


In [7]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=False)
    
    return model

0:	test: 0.9099922	best: 0.9099922 (0)	total: 500ms	remaining: 16m 38s
1:	test: 0.9227177	best: 0.9227177 (1)	total: 1.02s	remaining: 17m 3s
2:	test: 0.9295566	best: 0.9295566 (2)	total: 1.59s	remaining: 17m 38s
3:	test: 0.9318209	best: 0.9318209 (3)	total: 2.1s	remaining: 17m 29s
4:	test: 0.9333774	best: 0.9333774 (4)	total: 2.68s	remaining: 17m 50s
5:	test: 0.9334497	best: 0.9334497 (5)	total: 3.31s	remaining: 18m 18s
6:	test: 0.9342957	best: 0.9342957 (6)	total: 3.91s	remaining: 18m 32s
7:	test: 0.9343396	best: 0.9343396 (7)	total: 4.49s	remaining: 18m 38s
8:	test: 0.9343813	best: 0.9343813 (8)	total: 5.09s	remaining: 18m 45s
9:	test: 0.9349007	best: 0.9349007 (9)	total: 5.69s	remaining: 18m 53s
10:	test: 0.9349747	best: 0.9349747 (10)	total: 6.34s	remaining: 19m 7s
11:	test: 0.9349908	best: 0.9349908 (11)	total: 6.97s	remaining: 19m 15s
12:	test: 0.9354136	best: 0.9354136 (12)	total: 7.53s	remaining: 19m 10s
13:	test: 0.9356315	best: 0.9356315 (13)	total: 8.13s	remaining: 19m 13s
1

113:	test: 0.9404227	best: 0.9404227 (113)	total: 1m 4s	remaining: 17m 51s
114:	test: 0.9403765	best: 0.9404227 (113)	total: 1m 5s	remaining: 17m 51s
115:	test: 0.9404008	best: 0.9404227 (113)	total: 1m 5s	remaining: 17m 51s
116:	test: 0.9404265	best: 0.9404265 (116)	total: 1m 6s	remaining: 17m 50s
117:	test: 0.9404424	best: 0.9404424 (117)	total: 1m 7s	remaining: 17m 48s
118:	test: 0.9404634	best: 0.9404634 (118)	total: 1m 7s	remaining: 17m 47s
119:	test: 0.9405394	best: 0.9405394 (119)	total: 1m 8s	remaining: 17m 49s
120:	test: 0.9405084	best: 0.9405394 (119)	total: 1m 8s	remaining: 17m 50s
121:	test: 0.9404664	best: 0.9405394 (119)	total: 1m 9s	remaining: 17m 50s
122:	test: 0.9404731	best: 0.9405394 (119)	total: 1m 10s	remaining: 17m 48s
123:	test: 0.9404654	best: 0.9405394 (119)	total: 1m 10s	remaining: 17m 47s
124:	test: 0.9404561	best: 0.9405394 (119)	total: 1m 11s	remaining: 17m 45s
125:	test: 0.9404731	best: 0.9405394 (119)	total: 1m 11s	remaining: 17m 44s
126:	test: 0.9404791	

222:	test: 0.9411137	best: 0.9411166 (218)	total: 1m 57s	remaining: 15m 39s
223:	test: 0.9411052	best: 0.9411166 (218)	total: 1m 58s	remaining: 15m 38s
224:	test: 0.9411299	best: 0.9411299 (224)	total: 1m 58s	remaining: 15m 37s
225:	test: 0.9411259	best: 0.9411299 (224)	total: 1m 59s	remaining: 15m 35s
226:	test: 0.9411261	best: 0.9411299 (224)	total: 1m 59s	remaining: 15m 34s
227:	test: 0.9411273	best: 0.9411299 (224)	total: 2m	remaining: 15m 33s
228:	test: 0.9411356	best: 0.9411356 (228)	total: 2m	remaining: 15m 32s
229:	test: 0.9411209	best: 0.9411356 (228)	total: 2m	remaining: 15m 31s
230:	test: 0.9410949	best: 0.9411356 (228)	total: 2m 1s	remaining: 15m 30s
231:	test: 0.9410978	best: 0.9411356 (228)	total: 2m 1s	remaining: 15m 28s
232:	test: 0.9411353	best: 0.9411356 (228)	total: 2m 2s	remaining: 15m 27s
233:	test: 0.9411603	best: 0.9411603 (233)	total: 2m 2s	remaining: 15m 26s
234:	test: 0.9411981	best: 0.9411981 (234)	total: 2m 3s	remaining: 15m 25s
235:	test: 0.9412373	best: 0.

331:	test: 0.9413930	best: 0.9414086 (323)	total: 2m 47s	remaining: 13m 59s
332:	test: 0.9413890	best: 0.9414086 (323)	total: 2m 47s	remaining: 13m 58s
333:	test: 0.9414109	best: 0.9414109 (333)	total: 2m 47s	remaining: 13m 56s
334:	test: 0.9414346	best: 0.9414346 (334)	total: 2m 48s	remaining: 13m 55s
335:	test: 0.9414315	best: 0.9414346 (334)	total: 2m 48s	remaining: 13m 54s
336:	test: 0.9414391	best: 0.9414391 (336)	total: 2m 48s	remaining: 13m 53s
337:	test: 0.9414581	best: 0.9414581 (337)	total: 2m 49s	remaining: 13m 52s
338:	test: 0.9414464	best: 0.9414581 (337)	total: 2m 49s	remaining: 13m 50s
339:	test: 0.9414486	best: 0.9414581 (337)	total: 2m 49s	remaining: 13m 49s
340:	test: 0.9414686	best: 0.9414686 (340)	total: 2m 50s	remaining: 13m 48s
341:	test: 0.9414457	best: 0.9414686 (340)	total: 2m 50s	remaining: 13m 47s
342:	test: 0.9414206	best: 0.9414686 (340)	total: 2m 51s	remaining: 13m 46s
343:	test: 0.9414280	best: 0.9414686 (340)	total: 2m 51s	remaining: 13m 44s
344:	test: 0

440:	test: 0.9416647	best: 0.9417088 (428)	total: 3m 29s	remaining: 12m 21s
441:	test: 0.9416798	best: 0.9417088 (428)	total: 3m 29s	remaining: 12m 19s
442:	test: 0.9416776	best: 0.9417088 (428)	total: 3m 30s	remaining: 12m 19s
443:	test: 0.9416717	best: 0.9417088 (428)	total: 3m 30s	remaining: 12m 18s
444:	test: 0.9416725	best: 0.9417088 (428)	total: 3m 30s	remaining: 12m 17s
445:	test: 0.9416884	best: 0.9417088 (428)	total: 3m 31s	remaining: 12m 16s
446:	test: 0.9416899	best: 0.9417088 (428)	total: 3m 31s	remaining: 12m 15s
447:	test: 0.9416998	best: 0.9417088 (428)	total: 3m 31s	remaining: 12m 14s
448:	test: 0.9416952	best: 0.9417088 (428)	total: 3m 32s	remaining: 12m 13s
449:	test: 0.9417019	best: 0.9417088 (428)	total: 3m 32s	remaining: 12m 13s
450:	test: 0.9416829	best: 0.9417088 (428)	total: 3m 33s	remaining: 12m 12s
451:	test: 0.9416913	best: 0.9417088 (428)	total: 3m 33s	remaining: 12m 12s
452:	test: 0.9416640	best: 0.9417088 (428)	total: 3m 34s	remaining: 12m 12s
453:	test: 0

...

1528:	test: 0.9425572	best: 0.9425871 (1468)	total: 10m 47s	remaining: 3m 19s
1529:	test: 0.9425610	best: 0.9425871 (1468)	total: 10m 48s	remaining: 3m 19s
1530:	test: 0.9425638	best: 0.9425871 (1468)	total: 10m 48s	remaining: 3m 18s
1531:	test: 0.9425742	best: 0.9425871 (1468)	total: 10m 49s	remaining: 3m 18s
1532:	test: 0.9425677	best: 0.9425871 (1468)	total: 10m 49s	remaining: 3m 17s
1533:	test: 0.9425647	best: 0.9425871 (1468)	total: 10m 49s	remaining: 3m 17s
1534:	test: 0.9425680	best: 0.9425871 (1468)	total: 10m 50s	remaining: 3m 16s
1535:	test: 0.9425624	best: 0.9425871 (1468)	total: 10m 50s	remaining: 3m 16s
1536:	test: 0.9425652	best: 0.9425871 (1468)	total: 10m 51s	remaining: 3m 16s
1537:	test: 0.9425555	best: 0.9425871 (1468)	total: 10m 51s	remaining: 3m 15s
1538:	test: 0.9425579	best: 0.9425871 (1468)	total: 10m 52s	remaining: 3m 15s
1539:	test: 0.9425495	best: 0.9425871 (1468)	total: 10m 52s	remaining: 3m 14s
1540:	test: 0.9425544	best: 0.9425871 (1468)	total: 10m 53s	rema

1634:	test: 0.9425781	best: 0.9425871 (1468)	total: 11m 28s	remaining: 2m 33s
1635:	test: 0.9425850	best: 0.9425871 (1468)	total: 11m 28s	remaining: 2m 33s
1636:	test: 0.9425780	best: 0.9425871 (1468)	total: 11m 28s	remaining: 2m 32s
1637:	test: 0.9425789	best: 0.9425871 (1468)	total: 11m 29s	remaining: 2m 32s
1638:	test: 0.9425552	best: 0.9425871 (1468)	total: 11m 29s	remaining: 2m 31s
1639:	test: 0.9425638	best: 0.9425871 (1468)	total: 11m 30s	remaining: 2m 31s
1640:	test: 0.9425662	best: 0.9425871 (1468)	total: 11m 30s	remaining: 2m 31s
1641:	test: 0.9425614	best: 0.9425871 (1468)	total: 11m 31s	remaining: 2m 30s
1642:	test: 0.9425519	best: 0.9425871 (1468)	total: 11m 31s	remaining: 2m 30s
1643:	test: 0.9425587	best: 0.9425871 (1468)	total: 11m 31s	remaining: 2m 29s
1644:	test: 0.9425578	best: 0.9425871 (1468)	total: 11m 32s	remaining: 2m 29s
1645:	test: 0.9425699	best: 0.9425871 (1468)	total: 11m 32s	remaining: 2m 28s
1646:	test: 0.9425600	best: 0.9425871 (1468)	total: 11m 33s	rema

1740:	test: 0.9425465	best: 0.9426037 (1693)	total: 12m 28s	remaining: 1m 51s
1741:	test: 0.9425458	best: 0.9426037 (1693)	total: 12m 29s	remaining: 1m 50s
1742:	test: 0.9425480	best: 0.9426037 (1693)	total: 12m 29s	remaining: 1m 50s
1743:	test: 0.9425314	best: 0.9426037 (1693)	total: 12m 30s	remaining: 1m 50s
1744:	test: 0.9425389	best: 0.9426037 (1693)	total: 12m 30s	remaining: 1m 49s
1745:	test: 0.9425535	best: 0.9426037 (1693)	total: 12m 30s	remaining: 1m 49s
1746:	test: 0.9425386	best: 0.9426037 (1693)	total: 12m 31s	remaining: 1m 48s
1747:	test: 0.9425406	best: 0.9426037 (1693)	total: 12m 31s	remaining: 1m 48s
1748:	test: 0.9425478	best: 0.9426037 (1693)	total: 12m 31s	remaining: 1m 47s
1749:	test: 0.9425563	best: 0.9426037 (1693)	total: 12m 32s	remaining: 1m 47s
1750:	test: 0.9425454	best: 0.9426037 (1693)	total: 12m 32s	remaining: 1m 47s
1751:	test: 0.9425630	best: 0.9426037 (1693)	total: 12m 32s	remaining: 1m 46s
1752:	test: 0.9425655	best: 0.9426037 (1693)	total: 12m 33s	rema

1846:	test: 0.9425899	best: 0.9426156 (1825)	total: 13m 11s	remaining: 1m 5s
1847:	test: 0.9425854	best: 0.9426156 (1825)	total: 13m 11s	remaining: 1m 5s
1848:	test: 0.9425910	best: 0.9426156 (1825)	total: 13m 11s	remaining: 1m 4s
1849:	test: 0.9425827	best: 0.9426156 (1825)	total: 13m 12s	remaining: 1m 4s
1850:	test: 0.9425827	best: 0.9426156 (1825)	total: 13m 12s	remaining: 1m 3s
1851:	test: 0.9425911	best: 0.9426156 (1825)	total: 13m 12s	remaining: 1m 3s
1852:	test: 0.9426190	best: 0.9426190 (1852)	total: 13m 13s	remaining: 1m 2s
1853:	test: 0.9426196	best: 0.9426196 (1853)	total: 13m 13s	remaining: 1m 2s
1854:	test: 0.9426275	best: 0.9426275 (1854)	total: 13m 14s	remaining: 1m 2s
1855:	test: 0.9426328	best: 0.9426328 (1855)	total: 13m 14s	remaining: 1m 1s
1856:	test: 0.9426364	best: 0.9426364 (1856)	total: 13m 14s	remaining: 1m 1s
1857:	test: 0.9426285	best: 0.9426364 (1856)	total: 13m 15s	remaining: 1m
1858:	test: 0.9426251	best: 0.9426364 (1856)	total: 13m 15s	remaining: 1m
1859:

1953:	test: 0.9426330	best: 0.9426442 (1908)	total: 13m 54s	remaining: 19.7s
1954:	test: 0.9426278	best: 0.9426442 (1908)	total: 13m 55s	remaining: 19.2s
1955:	test: 0.9426190	best: 0.9426442 (1908)	total: 13m 55s	remaining: 18.8s
1956:	test: 0.9426241	best: 0.9426442 (1908)	total: 13m 55s	remaining: 18.4s
1957:	test: 0.9426251	best: 0.9426442 (1908)	total: 13m 56s	remaining: 17.9s
1958:	test: 0.9426196	best: 0.9426442 (1908)	total: 13m 56s	remaining: 17.5s
1959:	test: 0.9426294	best: 0.9426442 (1908)	total: 13m 56s	remaining: 17.1s
1960:	test: 0.9426293	best: 0.9426442 (1908)	total: 13m 57s	remaining: 16.6s
1961:	test: 0.9426321	best: 0.9426442 (1908)	total: 13m 57s	remaining: 16.2s
1962:	test: 0.9426301	best: 0.9426442 (1908)	total: 13m 57s	remaining: 15.8s
1963:	test: 0.9426324	best: 0.9426442 (1908)	total: 13m 58s	remaining: 15.4s
1964:	test: 0.9426252	best: 0.9426442 (1908)	total: 13m 58s	remaining: 14.9s
1965:	test: 0.9426363	best: 0.9426442 (1908)	total: 13m 58s	remaining: 14.5s

In [53]:
ndcg_score(test.get_label().reshape(-1,5),model_yeti.predict(test).reshape(-1,5))

0.8851408289047646

In [76]:
%%timeit
import json
with open('../dataset/ranking_test.jsonl') as f:
    test_data = [json.loads(line) for line in f]
    
post_texts = []
comments_texts = []
total_texts = []
scores = []
groups = []
for i,example in enumerate(test_data):
    for comment in example['comments']:
        post_texts.append(example['text'])
        comments_texts.append(comment['text'])
test_dataframe = pd.DataFrame([post_texts,comments_texts]).T
preds = model_yeti.predict(test_dataframe).reshape(-1,5)
ranks = 4 - np.argsort(preds[:,:])
for i,example in enumerate(test_data):
    for j,comment in enumerate(example['comments']):
        comment['score'] = int(ranks[i,j])

2.38 s ± 79.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
len(test_data)

14004

In [73]:
import gzip
import json

def dicts_to_jsonl(data_list: list, filename: str, compress: bool = False) -> None:
    """
    Method saves list of dicts into jsonl file.
    :param data: (list) list of dicts to be stored,
    :param filename: (str) path to the output file. If suffix .jsonl is not given then methods appends
        .jsonl suffix into the file.
    :param compress: (bool) should file be compressed into a gzip archive?
    """
    sjsonl = '.jsonl'
    sgz = '.gz'
    # Check filename
    if not filename.endswith(sjsonl):
        filename = filename + sjsonl
    # Save data
    
    if compress:
        filename = filename + sgz
        with gzip.open(filename, 'w') as compressed:
            for ddict in data_list:
                jout = json.dumps(ddict) + '\n'
                jout = jout.encode('utf-8')
                compressed.write(jout)
    else:
        with open(filename, 'w') as out:
            for ddict in data_list:
                jout = json.dumps(ddict) + '\n'
                out.write(jout)

In [74]:
dicts_to_jsonl(test_data,'output_model.jsonl')